In [ ]:
import requests
import pandas as pd
import json,os

#sharepoint dev site
tenant_id = os.getenv('TENANT_ID')
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')

# Token endpoint
url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

# Get token for Microsoft Graph
data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
    'scope': 'https://graph.microsoft.com/.default'
}

token_response = requests.post(url, data=data)
access_token = token_response.json()['access_token']

veeva_headers = {
    'Authorization': '10C3ADA626AFC05741F0E77BA01E99EE3A60BDAB3E71DCAEA6D533A58B8A5097856FFFF2EB11C08AC42914D65B7E8DE37FA786A39B349002528F638E578AB911',  # Add your token
    'Content-Type': 'application/x-www-form-urlencoded',
}
sp_headers = {
        'Authorization': f'Bearer {access_token}',  
        'Content-Type': 'application/octet-stream'
        }
LOCAL_DOWNLOAD_PATH = './'

In [62]:
def get_lookup(specnumber):
    specnumber = 'V4A00000001L867'
    params = {
        "q":f"select specification_number__c from formulation__v where id = '{specnumber}'"
    }
    base_url = 'https://sb-envu-deployment.veevavault.com/api/v24.2/query'
    response = requests.post(url=base_url, headers=veeva_headers, params=params).json()
    return response['data'][0]['specification_number__c']


In [332]:
document_url = 'https://sb-envu-deployment.veevavault.com/api/v24.2/objects/documents'

start = 0
limit = 200 

response = requests.get(document_url, headers=veeva_headers)
document_response_json = response.json()

# Process documents
document_response_json["documents"] = document_response_json["documents"][0:2]
print(f"length:{len(document_response_json['documents'])}")
for doc_fields in document_response_json["documents"]:
    file_id = doc_fields['document']['id']
    file_name = doc_fields['document']['filename__v']
    lifecyclestate = doc_fields['document']['status__v']
    subtype = doc_fields['document']['subtype__v']
    actual_specnumber = get_lookup(doc_fields['document']['specification_main_version__c'])

    # Download file from veeva
    file_download_url = f"https://sb-envu-deployment.veevavault.com/api/v24.2/objects/documents/{file_id}/file"
    file_response = requests.get(url=file_download_url,headers=veeva_headers)
    drive_id = 'b!XF8SojBiyEyRIG_EGLeYyI2648HNTIRCons_up6eLkQFVzl26doLTKtAm1ik_kXe'
    folder_name = 'veeva_documents'
    folder_name_encoded = folder_name.replace(" ", "%20") if folder_name else ""
    sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/octet-stream'
    }

    upload_url  = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{folder_name_encoded}/{file_name}:/content"
    upload_response  = requests.put(upload_url , headers=sp_headers, data=file_response.content)

    if upload_response.status_code not in [200, 201]:
        raise Exception(f"Failed to upload file to SharePoint: {upload_response.text}")

    uploaded_file_info = upload_response.json()
    item_id = uploaded_file_info["id"]  # This is the SharePoint file ID

    update_body = {
        "specification_x002b_mainversion": actual_specnumber,
        "Lifecyclestate":lifecyclestate,
        "Subtype":subtype # <-- column internal name
    }
    update_headers={
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json'   # must be JSON
}
    update_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/listItem/fields"
    update_resp = requests.patch(update_url, headers=update_headers, json=update_body)

    if update_resp.status_code == 200:
        print(f"File '{file_name}' uploaded and metadata updated successfully.")
    else:
        print(f"File uploaded but metadata update failed: {update_resp.text}")


length:2
File 'Final report_KLN_ENVU_2025-4.pdf' uploaded and metadata updated successfully.
File 'Hello everyone.docx' uploaded and metadata updated successfully.


In [63]:
file_download_url = f"https://sb-envu-deployment.veevavault.com/api/v24.2/objects/documents/9801/file"
file_name = 'Label_Albania_Finnish111.pdf'
file_response = requests.get(url=file_download_url,headers=veeva_headers)

print(f"file_response.statuscode:{file_response.status_code}")
drive_id = 'b!XF8SojBiyEyRIG_EGLeYyI2648HNTIRCons_up6eLkQFVzl26doLTKtAm1ik_kXe'
folder_name = 'veeva_documents'
folder_name_encoded = folder_name.replace(" ", "%20") if folder_name else ""
# update_headers={
#     'Authorization': f'Bearer {access_token}',
#     'Content-Type': 'application/json'   # must be JSON
# }

upload_url  = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/{folder_name_encoded}/{file_name}:/content"
upload_response  = requests.put(upload_url , headers=sp_headers, data=file_response.content)

file_response.statuscode:200


In [64]:
upload_response.content

b'{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#drives(\'b%21XF8SojBiyEyRIG_EGLeYyI2648HNTIRCons_up6eLkQFVzl26doLTKtAm1ik_kXe\')/root/$entity","@microsoft.graph.downloadUrl":"https://envu.sharepoint.com/sites/S3-WhatsappIntegration/_layouts/15/download.aspx?UniqueId=edb0b835-e943-4285-ac2d-83c46815359d&Translate=false&tempauth=v1.eyJzaXRlaWQiOiJhMjEyNWY1Yy02MjMwLTRjYzgtOTEyMC02ZmM0MThiNzk4YzgiLCJhcHBfZGlzcGxheW5hbWUiOiJTaGFyZVBvaW50IERldiBzaXRlIiwibmFtZWlkIjoiYWVkMzg3OGMtMDVhOC00OGVhLTlkZTgtODY0NDM0ZTMxYjIzQGM0ZGVkYjc0LWQ5MTYtNGVmNC1iNmI1LWFmODBjNTllOTc0MiIsImF1ZCI6IjAwMDAwMDAzLTAwMDAtMGZmMS1jZTAwLTAwMDAwMDAwMDAwMC9lbnZ1LnNoYXJlcG9pbnQuY29tQGM0ZGVkYjc0LWQ5MTYtNGVmNC1iNmI1LWFmODBjNTllOTc0MiIsImV4cCI6IjE3NjA1OTc5NzMifQ.CkAKDGVudHJhX2NsYWltcxIwQ1BlUXdzY0dFQUFhRm1GMmQxRmxTemhKY1ZWRGNHNXRNVzl1YzI5VlFVRXFBQT09CjIKCmFjdG9yYXBwaWQSJDAwMDAwMDAzLTAwMDAtMDAwMC1jMDAwLTAwMDAwMDAwMDAwMAoKCgRzbmlkEgI2NBILCPiEgePmoMY-EAUaDTIwLjE5MC4xNjMuMjkqLEtIRDMyZTZsVWFYYWV1UlNTZjNqU3VlWE8wRlU4d2RCa

File Endpoint

In [32]:

update_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root:/veeva_documents:/children"
sp_headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/octet-stream'
    }

document_info = []
upload_response  = requests.get(update_url , headers=sp_headers)
print(json.dumps(upload_response.json(),indent=3))
# uploaded_file_info = upload_response.json()
# # print(uploaded_file_info)
# # print(json.dumps(uploaded_file_info['value'],indent=3))
# for upload in uploaded_file_info['value']:
#     document_info.append({
#         'id': upload['id'],
#         'FileName': upload['name'],
#         'webUrl': upload['webUrl']
#     })

# document_df = pd.DataFrame(document_info)
# len(document_df)



{
   "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#Collection(driveItem)",
   "value": [
      {
         "@microsoft.graph.downloadUrl": "https://envu.sharepoint.com/sites/S3-WhatsappIntegration/_layouts/15/download.aspx?UniqueId=7b52695f-0f7f-4867-af8d-fa343674832b&Translate=false&tempauth=v1.eyJzaXRlaWQiOiJhMjEyNWY1Yy02MjMwLTRjYzgtOTEyMC02ZmM0MThiNzk4YzgiLCJhcHBfZGlzcGxheW5hbWUiOiJJbmRpYVByb2plY3RzVG9kYXkiLCJuYW1laWQiOiI2YThhOTlmNi0xOWMyLTRhMTEtOTk2OC0xMmU4M2NlMWJhZDRAYzRkZWRiNzQtZDkxNi00ZWY0LWI2YjUtYWY4MGM1OWU5NzQyIiwiYXVkIjoiMDAwMDAwMDMtMDAwMC0wZmYxLWNlMDAtMDAwMDAwMDAwMDAwL2VudnUuc2hhcmVwb2ludC5jb21AYzRkZWRiNzQtZDkxNi00ZWY0LWI2YjUtYWY4MGM1OWU5NzQyIiwiZXhwIjoiMTc2MDU0MDA0MyJ9.CkAKDGVudHJhX2NsYWltcxIwQ1BLK3ZzY0dFQUFhRmtSNlVYcEZOV2RpU0RCTGNqSkpORzh0UlUxYVFVRXFBQT09CjIKCmFjdG9yYXBwaWQSJDAwMDAwMDAzLTAwMDAtMDAwMC1jMDAwLTAwMDAwMDAwMDAwMAoKCgRzbmlkEgI2NBILCK6ztNiK_8U-EAUaDDQwLjEyNi4zNS44OCosUDkxblFydmpnbXVYMU0zSmpHTTdKWXFMaXlxejRmVXl2eWdLaHBhcW0xMD0wkAE4AUIQoc-lMTewAOAjwe8

MetaData of Files

In [40]:
# ids = document_df['id'].to_list()
# meta_data = pd.DataFrame()

# for item_id in ids:
item_id = '01SDPKBONN3OJHFE2IE5CJL2XV6E5P2CWN'
drive_id = 'b!XF8SojBiyEyRIG_EGLeYyI2648HNTIRCons_up6eLkQFVzl26doLTKtAm1ik_kXe'
fields_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{item_id}/listItem/fields"
upload_response  = requests.get(fields_url, headers=sp_headers).json()

print(json.dumps(upload_response, indent=3))

# temp_responce = [{
#     'id': item_id,
#     "Specification_number": upload_response.get('specification_x002b_mainversion', None),
#     "Lifecyclestate":upload_response.get('Lifecyclestate',None),
    
# }]

# temp_df = pd.DataFrame(temp_responce)

# Concatenate along rows, reset index
# meta_data = pd.concat([meta_data, temp_df], ignore_index=True)

# meta_data

{
   "@odata.context": "https://graph.microsoft.com/v1.0/$metadata#drives('b%21XF8SojBiyEyRIG_EGLeYyI2648HNTIRCons_up6eLkQFVzl26doLTKtAm1ik_kXe')/items('01SDPKBONN3OJHFE2IE5CJL2XV6E5P2CWN')/listItem/fields/$entity",
   "@odata.etag": "\"7292dbad-4893-4427-95ea-f5f13afd0acd,16\"",
   "Modified": "2025-10-15T14:05:50Z",
   "FileLeafRef": "EA_NewReq00077_LOA Specific studies Indaziflam_Specticle G Herbicide (Spec 102000029955)_Envu Austra.pdf",
   "specification_x002b_mainversion": "102000001388",
   "Lifecyclestate": "Released",
   "Subtype": "LoA (Letter of Access)",
   "Documentdescription": "EA_NewReq00077_LOA Specific studies Indaziflam_Specticle G Herbicide (Spec 102000029955)_Envu Austra.pdf",
   "rootspec": "102000001388",
   "specificationminorversion": "102000001388-0",
   "documentid0": 879.0,
   "id": "303",
   "ContentType": "Document",
   "Created": "2025-10-15T13:43:10Z",
   "AuthorLookupId": "1073741822",
   "EditorLookupId": "25",
   "_CheckinComment": "",
   "LinkFilenam

In [ ]:
merge_df = pd.merge(document_df,meta_data,on='id',how='left')

In [ ]:
merge_df

,id,FileName,webUrl,Specification_number,Lifecyclestate
0,01SDPKBOJJRQZ6HIY3IZGJX5NDRR66JMCD,Final report_KLN_ENVU_2025-4.pdf,https://envu.sharepoint.com/sites/S3-WhatsappI...,102000000534-02,Released
1,01SDPKBOK7GI66V6YOQFE24XMGJ53Z2KFJ,Hello everyone.docx,https://envu.sharepoint.com/sites/S3-WhatsappI...,102000000534-02,In Review
2,01SDPKBOP73XAVFMC2VNGK2FGSVPQZ5XBH,mini contracat.docx,https://envu.sharepoint.com/sites/S3-WhatsappI...,1032344125-09,Open


In [ ]:
with open('./failed_uploads.json', 'r') as f:
    data = json.load(f)  # parse JSON from file

print(len(data))
# print(json.dumps(data, indent=3))  # pre

88
